# **Progetto DeepLearning**
Lorenzo Gardini

# Implementazione Manuale

## **Project goals**
Lo scopo del progetto è quello di studiare, addestrare e comparare algoritmi di Reinforcement Learning applicati al gioco Pong dell'Atari (https://en.wikipedia.org/wiki/Pong). Gli algoritmi utilizzati sono PPO, A2C e DQN con aggiunte per migliorarne le prestazioni (Dueling DQN, Prioritized Experience Replay, Double DQN).

## **Pong**
Pong è un videogioco bidimensionale che simula il ping-pong. I due giocatori possono muovere la rispettiva racchetta (simulata da una barra bianca) che può muovere verticalmente lungo un lato dello schermo per colpire la pallina (simulata da un piccolo quadrato) per rispedirla indietro. Se la pallina supera la racchetta di un giocatore il suo avversario guadagna un punto. La palla si muove con velocità costante e può rimbalzare sui lati superiore e inferiore dello schermo.

<p align="center">
  <img src="https://miro.medium.com/max/320/1*P4l2XZUffcJfJQjQ125wSw.gif" width="300"/>
</p>

## **Installazione librerie**
- stable-baselines3 è una libreria open source di reinforcement learning. Da questa utiliziamo dei wrapper per l'environment.
- ale-py serve per i permessi con i giochi dell'Atari.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import random
import time
import statistics
import cv2
import uuid
from base64 import b64encode
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import deque
from gym import spaces
from abc import abstractmethod, ABC
import pandas as pd
import pickle
import math

## Utility

- `print_episode_info`: stampa lo stato corrente dell'apprendimento della rete. 
- `save_model`: salva il modello (`dqn_action_model`, `dqn_target_model`, `train_rewards`, `epsilon`, `beta`, `replay_memory`) nel `path` specificato
- `load_model`: carica il modello (`dqn_action_model`, `dqn_target_model`, `train_rewards`, `epsilon`, `beta`, `replay_memory`) dal `path` specificato per riprendere l'addestramento 

In [4]:
def print_episode_info(episode_number, 
                       step_count, 
                       train_step_count, 
                       episode_start_epsilon, 
                       epsilon,
                       episode_start_beta,
                       beta,
                       episode_elapsed_time,
                       episode_avg_step_time,
                       episode_reward,
                       moving_avg_reward):
  print(f'Episode: {episode_number} Steps: {step_count}[{train_step_count}]',
        f'Epsilon: [{episode_start_epsilon:.3f}->{epsilon:.3f}]',
        f'Beta: [{episode_start_beta:.3f}->{beta:.3f}]',
        f'Time: {episode_elapsed_time:.1f}s[{episode_avg_step_time:.2f}s]',  
        f'Total reward: {episode_reward}[{moving_avg_reward:.1f}]')
  
def save_model(path,
               dqn_action_model, 
               dqn_target_model, 
               train_rewards, 
               epsilon, 
               beta, 
               replay_memory):
  print('saving...')
  dynamic_path = lambda x: path + '/' + x
  dqn_action_model.save(dynamic_path('dqn_action_model')) 
  dqn_target_model.save(dynamic_path('dqn_target_model'))
  pd.Series(train_rewards).to_csv(dynamic_path('train_rewards.csv'), index=False)
  other_data = {
      'beta':beta,
      'epsilon':epsilon,
      'replay_memory':replay_memory
  }
  with open(dynamic_path('other_data.bin'), 'wb+') as data:
    pickle.dump(other_data, data)
  print('saved!')

def load_model(path):
  try:
    data = {}
    data['dqn_action_model'] = keras.models.load_model(path + '/dqn_action_model')
    data['dqn_target_model'] = keras.models.load_model(path + '/dqn_target_model')
    data['train_rewards'] = list(pd.read_csv(path + '/train_rewards.csv', squeeze=True, dtype=np.int8))
    with open(path + '/other_data.bin', 'rb') as other_data:
      data.update(pickle.load(other_data))
    print('previous data loaded')
    return data
  except Exception as e:
    print(f"ERROR: {e}")
    print('Data not loaded')
    return {}

## Specifiche dell'environment


Come environment per l'addestramento abbiamo usato la libreria Gym di Open AI (https://www.gymlibrary.dev/environments/atari/pong/#pong). In questa versione di pong la racchetta sinistra è controllata dal computer mentre la racchetta destra può essere mossa manualmente. Sarà proprio questa racchetta che verrà controllata dagli algoritmi. Le azioni   possibili che si possono effettuare sono:

In [5]:
env_name = 'PongNoFrameskip-v4'
env = gym.make(env_name)

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


Di seguito le azioni possibili in gioco e il loro significati

In [6]:
list(enumerate(env.get_action_meanings()))

[(0, 'NOOP'),
 (1, 'FIRE'),
 (2, 'RIGHT'),
 (3, 'LEFT'),
 (4, 'RIGHTFIRE'),
 (5, 'LEFTFIRE')]

le dimensioni dello spazio dell'environment (le dimensioni del tensore restituito)

In [7]:
env.observation_space.shape

(210, 160, 3)

I valori minimi e massimi possibili per le osservazioni (essendo un'immagine RGB i valori sono quelli dei pixel per ogni colore)

In [8]:
env.observation_space.low
env.observation_space.high

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

## Environment preprocessing


In questa parte viene effettuato il preprocessing dell'environment. Questo viene fatto per migliorare e stabilizzare l'apprendimento della rete. Le varie trasformazioni che vengono applicati all'env sono:
- `FrameStack`: prende in ingresso `k` che corrisponde quanti frame impilare per dare alla rete il senso del movimento. L'ultimo stato (quindi lo stato corrente) si trova alla k-esima posizione del tensore.
- `StickyActionEnv`: con una probabilità di `action_repeat_probability` l'environment ignora l'azione da eseguire e ripete la precedente azione. Questo permette alla rete (anche con un ϵ molto basso) di mantenere la rete stocastica e più imprevedibile in modo da migliorare l'addestramento (https://arxiv.org/pdf/1709.06009.pdf)
- `PreprocessImage`: preprocessa le immagini in modo da ridurre la complessità e ridurre il tempo d'addestramento. L' immagine viene tagliata usando i crops specificati con `crops` che corrisponde ad una tupla (top_crop, right_crop, bottom_crop, left_crop) e poi ridimensionata con le dimensioni specificati `shape` (width, height)
- `SkipEnv`: Skippa `skip` frames (per accellerare l'apprendimento).


In [9]:
class FrameStack(gym.Wrapper):
  def __init__(self, env, k):
    super().__init__(env)
    self.k = k
    self.frames = deque([], maxlen=k)
    shp = (*env.observation_space.shape, k)
    self.observation_space = spaces.Box(low=0, high=255, shape=shp, dtype=env.observation_space.dtype)

  def reset(self):
    ob = self.env.reset()
    for _ in range(self.k):
        self.frames.append(ob)
    return self._get_ob()

  def step(self, action):
    ob, reward, done, info = self.env.step(action)
    self.frames.append(ob)
    return self._get_ob(), reward, done, info

  def _get_ob(self):
    assert len(self.frames) == self.k
    return np.stack(self.frames, axis=2)

class StickyActionEnv(gym.Wrapper):
  def __init__(self, env: gym.Env, action_repeat_probability: float) -> None:
    super().__init__(env)
    self.action_repeat_probability = action_repeat_probability

  def reset(self, **kwargs):
    self._sticky_action = 0  # NOOP
    return self.env.reset(**kwargs)

  def step(self, action: int):
    if self.np_random.random() >= self.action_repeat_probability:
      self._sticky_action = action
    return self.env.step(self._sticky_action)

class SkipEnv(gym.Wrapper):
  def __init__(self, env: gym.Env, skip: int = 4) -> None:
    super().__init__(env)
    self._skip = skip

  def step(self, action: int):
    total_reward = 0.0
    done = False
    for _ in range(self._skip):
      obs, reward, done, info = self.env.step(action)
      total_reward += reward
      if done:
          break
    return obs, total_reward, done, info

class PreprocessImageEnv(gym.ObservationWrapper):
  def __init__(self, env: gym.Env, shape=(84,84), crops=(None, None, None, None)):
    super().__init__(env)
    self.shape = shape
    self.crops = crops
    self.observation_space = spaces.Box(
        low=0, high=255, shape=shape, dtype=env.observation_space.dtype
    )
    obs_dims = self.env.observation_space
    self.height = obs_dims.shape[0]
    self.width = obs_dims.shape[1]
    self.obs = np.empty((self.height, self.width), dtype=np.uint8)

  def observation(self, frame: np.ndarray) -> np.ndarray:
    self.env.ale.getScreenGrayscale(self.obs)
    height, width = self.obs.shape

    top_crop, right_crop, bottom_crop, left_crop = self.crops
    top_crop = 0 if top_crop is None else top_crop
    right_crop = width if right_crop is None else right_crop
    bottom_crop = height if bottom_crop is None else bottom_crop
    left_crop = 0 if left_crop is None else left_crop

    frame = self.obs[left_crop:right_crop, top_crop:bottom_crop]
    frame = cv2.resize(frame, self.shape, interpolation=cv2.INTER_AREA)
    int_image = np.asarray(frame, dtype=np.uint8)
    return frame

Questa funzione permette di creare l'environment specificato applicando i wrappers elencati prima

In [10]:
def make_env(env_name, stacked_frames, frame_skip, action_repeat_probability, shape, crops):
  env = gym.make(env_name)
  env = StickyActionEnv(env, action_repeat_probability)
  env = SkipEnv(env, skip=frame_skip)
  env = PreprocessImageEnv(env, shape=shape, crops=crops)
  env = FrameStack(env, k=stacked_frames)
  return env

## Struttura della rete (Dueling DQN)

Rete introdotta nel 2015 da DeepMind (https://arxiv.org/pdf/1511.06581.pdf).
Per capire come funziona bisogna prima considerare che il Q-Value di una coppia $(s, a)$ può essere espressa come $Q(s, a) = V(s) + A(s, a)$ dove $V(s)$ corrisponde al valore dell'azione $s$ mentre $A(s, a)$ corrisponde al _**vantaggio**_ di effettuare l'azione $s$ nello stato $a$ rispetto a tutte le altre possibili azioni in quello stato. Inoltre, il valore di uno stato è uguale al Q-Value della migliore azione $a^*$ per quello
stato (poiché assumiamo che la politica ottimale scelga l'azione migliore), quindi $V(s) = Q(s, a^*)$, il che implica che $A(s, a^*) = 0$. Nel Dueling DQN il modello stima sia
il valore dello stato e il vantaggio di ogni possibile azione. Dal momento che l'azione migliore dovrebbe avere un vantaggio pari a 0, il modello sottrae il massimo vantaggio previsto da tutti i vantaggi previsti. Questo deriva dal fatto che la somma di $V$ e $A$ è "non identificabile" in quanto dato $Q$ non è possibile recuperare $V$ e $A$ in modo univoco. Sottrarre il massimo di $A$ forza $Q$ ad essere uguale a $V$. 

Ecco come si configura la rete:
<p align="center">
  <img src='https://www.fromkk.com/images/ddqn_duel_dqn.png' margin='center'/>
</p>

Sopra la classica configurazione CNN della rete DQN.
Sotto la configurazione di una rete Dueling. L'ultimo layer convoluzonale viene appiattito con il layer `Flatten` e viene collegato con due layer `Dense` di 512 neuroni ciascuno. $A$ viene modellato con un layer di $n$ neuroni pari al numero di azioni possibili del gioco mentre $V$ è un singono neurone denso. Infine i due valori vengono combinati per calcolare i Q_values

In [11]:
def build_dqn(input_shape=(84, 84, 4),action_count=9):
    model=keras.Sequential(
            [
                layers.Input(shape=input_shape,name='Input'),
                layers.Conv2D(filters=32, kernel_size=8, strides=4,activation='relu',padding='valid',name='C1'),
                layers.Conv2D(filters=64, kernel_size=4,strides=2,activation='relu',padding='valid',name='C2'),
                layers.Conv2D(filters=64, kernel_size=3,strides=1,activation='relu',padding='valid',name='C3'),
                layers.Flatten(),
                layers.Dense(512, activation='relu',name='FC'),
                layers.Dense(units=action_count,activation='linear',name='Output')
            ]
        )
    return model

def build_dueling_dqn(input_shape=(84, 84, 4), action_count=9): 
  input = layers.Input(shape=input_shape, name='input')
  conv_1 = layers.Conv2D(filters=32, kernel_size=8, strides=4, padding='valid', activation='relu')(input)
  conv_2 = layers.Conv2D(filters=64, kernel_size=4, strides=2, padding='valid', activation='relu')(conv_1)
  conv_3 = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='valid', activation='relu')(conv_2)
  flatten = layers.Flatten()(conv_3)
  # branch state
  flatten_state = layers.Dense(512, activation='relu')(flatten)
  state = layers.Dense(1, activation='linear')(flatten_state)
  # branch advantages
  flatten_advantages = layers.Dense(512, activation='relu')(flatten)
  raw_advantages = layers.Dense(action_count,activation='linear')(flatten_advantages) 

  # action score - mean action score = centered action score
  advantages = raw_advantages - tf.reduce_max(raw_advantages, axis=1, keepdims=True)
  Q_values = state + advantages

  model = keras.Model(inputs=[input], outputs=[Q_values])
  return model

## Prioritized Experience Replay (PER)

Soluzione presentata da DeepMind nel 2015 per migliorare le prestazioni di DQN (https://arxiv.org/pdf/1511.05952.pdf). L'idea è quella che invece di campionare le esperienze in modo uniforme dal buffer di riproduzione vengono campionate le esperienze importanti più frequentemente. Le esperienze sono considerate "importanti" se possono condurre a rapidi progressi nell'apprendimento. Poiché i campioni saranno sbilanciati verso le esperienze importanti si compensa questa distorsione durante l'addestramento riducendo il peso delle esperienze in base alla loro importanza, altrimenti il modello si overfitterà le esperienze importanti. Per fare ciò si da loro un peso inferiore durante l'allenamento.



### **Calcolo delle probabilità**
La probabilità di campionamento di una trasizione i è:
$$ P(i) = \frac{p_i^\alpha}{\sum_k{p_k^\alpha}}$$

dove $\alpha$ è un iperparametro che determina la quantità di priorità utilizzata, con $\alpha=0$ corrispondente al caso uniforme. $p_i$ è la priorità.

Per fare in modo che la rete non overfitti si corregge il bias utilizzando dei pesi $w$ nella loss function:

$$ w_i = (N \cdot P(i))^{-\beta}$$

dove $N$ è il numero di esperienze nel buffer e $\beta$ è un iperparametro che controlla quanto compensare il bias (0 significa per niente, mentre 1 significa del tutto). Nel paper si usa $\beta = 0.4$ all'inizio dell'addestramento che aumenta linearmente fino a $\beta = 1$

I $w_i$ vengono normalizzati con $\frac{1}{max_i \; w_i}$ per stabilità.


### **Binary Segment Tree**
Innanzitutto, si può semplicemente implementare PER ordinando in base alla priorità. Ciò non sarà efficiente a causa di $O(n \cdot log(n))$ per l'inserimento e $O(n)$ per il campionamento.
Per effettuare un campionamento pesato in modo efficiente si utilizza una struttura dati chiamata **Binary Segment Tree**.

Viene usato per calcolare velocemente $\sum_k^i{p_k^a}$ e $min \; p_i^\alpha$ che serve per calcolare $\frac{1}{max_i \; w_i}$. Per effettuare questi calcoli occorrono un **SumTree** e un **MinTree**. Entrambi sono dei **Binary Segment Tree**.

Il Binary Segment Tree consente di effettuare gli inserimenti con $O(log(n)$ e i calcoli in $O(log(n))$, che sono molto più efficienti di $O(n \cdot log(n))$ e $O(n)$

Sia $x_i$ la lista di $N$ valori che si vogliono rappresentare. Sia $b_{i,j}$ il $j^{th}$ nodo della $i^{th}$ riga dell'albero binario. I figlio del nodo $b_{i,j}$ sono $b_{i+1,2j}$ $b_{i+1,2j+1}$. 

Le foglie in posizione $D = \lceil 1+ log_2N  \rceil$ hanno i valori di $x$. Ogni nodo mantiene la somma dei valori dei nodi figli. La radice quindi contiene la somma dell'intero array di valori. Il figlio sinistro e destro della radice mantengono la somma rispettivamente della prima metà e della seconda dell'array.

$$ b_{i,j} = \displaystyle\sum_{k=(j-1) * 2^{D-i} + 1} ^ {j*2^{D-i}} x^k$$

<p align="center">
  <img src='https://pylessons.com/media/Tutorials/Reinforcement-learning-tutorial/CartPole-PER/SumTree.png' width='800' />
</p>

Il numero di nodi in ogni riga $i$ sono:

$$N_i = \left\lceil\dfrac{N}{D-i+1}\right\rceil $$

Questo è equivalente alla somma di tutte le righe sotto $i$. Quindi basta usare un array per salvare tutto l'albero dove 

$$b_{i,j} \rightarrow a_{N_i + j}$$

I figli di $a_i$ sono $a_{2i}$ e $a_{2i +1}$ tali che:

$$a_i = a_{2i} + a_{2i +1}$$

N.B. Gli indici iniziano da 1 e non da 0 (per gestire facilmente l'albero)

N.B. Gli stessi ragionamenti vengono effettuati per creare un **Min Heap**. In questo caso al posto dlla somma si usa l'operatore minimo.






### **Come si fa sampling?**
1. Si genera un numero random tra 0 e la radice in modo uniforme.
2. Si confronta il valore con il nodo di sinistra, Se il valore è minore si scende lungo il nodo e si ripete il punto 2. Se il valore è maggiore allora si scende al nodo di destra sottraendo il valore del nodo di sinistra e si ripete il punto 2.
3. Si raggiunge una foglia. Questo valore corrisponde ad una priorità di una transizione.
4. Si ottiene l'indice della transizione sottraendo all'indice dell'albero la capacità del vettore.
6. Si ripetono i passi da 1. a 6. finchè non si ottiene il mini-batch della grandezza desiderata.
5. Vengono restituiti al chiamante il mini-batch delle transizioni, i corrispettivi indici (verranno utilizzati per aggiornare le priorità delle transizioni dopo la backpropagation) e i corrispettivi $w_i$ (per eliminare il bias durante la backpropagation)
<p align="center">
  <img src='http://www.sefidian.com/wp-content/uploads/2022/05/image-15.png' width='500' />
</p>

### **Inserimento nuova transizione**

La nuova transizione viene aggiunta con il valore di priorità impostato come iperparametro, solitamente = 1 e poi vengono aggiornati i due alberi propagando il valore dalle foglie al nodo radice e applicando la funzione specifica dell'albero (somma e minimo).  

### **Utilizzo dei $w_i$ e aggiornamento delle priorità**

Dato che il sampling pesato crea un bias allora l'aggiornamento della rete viene pesato utilizzando i $w_i$. Questi valori vengono moltiplicati al gradiente prima di effettuare la backpropagation.

$$\Delta \leftarrow \Delta + w_i \cdot \delta_j \cdot \nabla_\theta Q(S_{j-1}, A_{j-1})$$

con $ j $ indice che scorre i $k$ elementi di un minibatch

I $\delta_j$ vengono utilizzati per aggiornare le priorità: se una transizione ha generato poco errore allora la sua priorità deve essere diminuita, viceversa se ha generato un errore grande. Le priorità si aggiornano utilizzando gli indici ritornati durante la fase di sampling. L'aggiornamento si effettua cosi:
 
 $$p_i = |\delta_i| + \epsilon $$
 Dove $\delta_i$ è l'errore commesso dalla i-esima transizione e $\epsilon$ è un iperarametro (solitamente messo a 0.1 o 0.01) che crea una base per fare in modo che la probabilità di campionare una transizione non diventi mai 0.

### **Implementazione**

Implementazione della classe astratta di una ReplayMemory. 
- `append`: aggiunge una transizione alla reply memory
- `sample_minibatch`: seleziona un mini-batch delle dimensioni `batch_size` e lo divide con `_split_minibatch`
- metodo statico privato `_split_minibatch`: splitta un batch di transitions in 5 batch `[state_batch, action_batch, reward_batch, new_state_batch, done_batch]` 

In [12]:
class ReplayMemory(ABC):
  @abstractmethod
  def __len__(self):
    pass
  @abstractmethod
  def append(self, transition):
    pass

  @abstractmethod
  def sample_minibatch(self, batch_size):
    pass

  @staticmethod
  def _split_minibatch(minibatch):
    state_batch = np.array([sample[0] for sample in minibatch])
    action_batch = np.array([sample[1] for sample in minibatch])
    reward_batch = np.array([sample[2] for sample in minibatch])
    new_state_batch = np.array([sample[3] for sample in minibatch])
    done_batch = np.array([sample[4] for sample in minibatch])

    return [state_batch,action_batch,reward_batch,new_state_batch,done_batch]

Implementazione della Replay Memory base. Implementata come una lista circolare perchè la classe deque è molto lenta in lettura per dimensioni molto grandi (è implementata come una doppia liked list)

In [13]:
class CircularReplayMemory(ReplayMemory):
  def __init__(self, max_size):
    self._buffer = np.empty(max_size, dtype=object)
    self.max_size = max_size
    self.index = 0
    self.size = 0

  def append(self, transition):
    self._buffer[self.index] = transition
    self.size = min(self.size + 1, self.max_size)
    self.index = (self.index + 1) % self.max_size

  def __getitem__(self,index):
    return self._buffer[index]
  
  def __len__(self):
    return self.size

  def sample_minibatch(self, batch_size):
    minibatch_indices = np.random.choice(range(self.size), size=batch_size)
    minibatch = [self._buffer[i] for i in minibatch_indices]
    return self._split_minibatch(minibatch)

Classe astratta per una generica Prioritized Replay Memory.

- `sample_minibatch`: ora il sampling del mini-batch riceve come parametro aggiuntivo $\beta$ per calcolare i $w_i$. Oltre ai minibatch di *state, reward, done, new_state* ritorna anche gli indici $i$ e i pesi $w_i$ corrispondenti alle transizioni selezionate
- `update_priorities`: funzione che aggiorna le priorità delle transizioni nelle posizioni `indexes`. `losses` sono i $\delta_i$ computati durante l'aggiornamento della rete.

In [14]:
class PrioritizedMemory(ReplayMemory):
  @abstractmethod
  def sample_minibatch(self, batch_size, beta):
    pass

  @abstractmethod
  def update_priorities(self, indexes, losses):
    pass

### Implementazione PER
Implementazione della PER memory utilizzando gli alberi per ottimizzare i calcoli

In [15]:
class TreePrioritizedReplayMemory(PrioritizedMemory):
  def __init__(self, capacity, alpha=0.6, error_offset=0.1):
    self._capacity = self._compute_power_2_capacity(capacity)
    self._alpha = alpha
    self._error_offset = error_offset
    # Maintain segment binary trees to take sum and find minimum
    self._priority_sum = np.zeros(2 * self._capacity)
    self._priority_min = np.full(2 * self._capacity, np.inf)
    self._data = np.full(self._capacity, None)
    # Current max priority, p, to be assigned to new transitions
    self._max_priority = 1.
    # We use cyclic buffers to store data, and next_idx keeps the index of the next empty slot
    self._next_idx = 0
    # Size of the buffer
    self._size = 0

  def __len__(self):
    return self._size

  def append(self, transition):
    # Get next available slot
    idx = self._next_idx
    # store in the queue
    self._data[idx] = transition
    # Increment next available slot
    self._next_idx = (idx + 1) % self._capacity
    # Calculate the size
    self._size = min(self._capacity, self._size + 1)
    # p_i ^ alpha, new samples get max_priority
    priority_alpha = self._max_priority ** self._alpha
    # Update the two segment trees for sum and minimum
    self._set_priority_min(idx, priority_alpha)
    self._set_priority_sum(idx, priority_alpha)

  def sample_minibatch(self, batch_size, beta):
    # to normalize weights we divide by 1/max_w for stability
    # the next two lines computes max_w
    # p_i ^ alpha / sum(p ^ alpha) => min prob = 1/min prob => the biggest fraction
    prob_min = self._min() / self._sum() # P(i)
    # (min P(i) * N) ^ (-beta) = (1/N * 1/P(i)) ^ beta => compute max w 
    max_weight = (prob_min * self._size) ** (-beta)

    # Initialize samples
    weights = np.zeros(batch_size, dtype=np.float32)
    # Get sample indices
    indices = np.array([self._find_prefix_sum_idx(p) for p in np.random.random(size=batch_size) * self._sum()])   

    for i in range(batch_size):
      idx = indices[i]
      # P(i) = p_i ^ alpha / (sum p^alpha)
      prob = self._priority_sum[idx + self._capacity] / self._sum()
      # (P(i) * N)^(-beta) = (1/N * 1/P(i)) ^ beta
      weight = (prob * self._size) ** (-beta)
      # Normalize by 1/max_weight which also cancels off the 1/N term
      weights[i] = weight / max_weight

    return (*self._split_minibatch(self._data[indices]), weights, indices)

  def update_priorities(self, indexes, losses):
    priorities = np.abs(losses) + self._error_offset
    # Set current max priority
    self._max_priority = max(self._max_priority, max(priorities))

    for idx, priority in zip(indexes, priorities):
      # Calculate p_i^alpha
      priority_alpha = priority ** self._alpha
      # Update the trees
      self._set_priority_min(idx, priority_alpha)
      self._set_priority_sum(idx, priority_alpha)

  def _compute_power_2_capacity(self, capacity):
    # convert capacity to a power of 2
    new_capacity = 1
    while new_capacity < capacity:
        new_capacity *= 2
    return new_capacity

  def _set_priority_min(self, idx, priority_alpha):
    # Leaf of the binary tree
    idx += self._capacity
    self._priority_min[idx] = priority_alpha
    # Update tree, by traversing along ancestors. Continue until the root of the tree.
    while idx >= 2:
      # Get the index of the parent node
      idx //= 2
      # Value of the parent node is the minimum of it's two children
      self._priority_min[idx] = min(self._priority_min[2 * idx], self._priority_min[2 * idx + 1])

  def _set_priority_sum(self, idx, priority):
    # Leaf of the binary tree
    idx += self._capacity
    # Set the priority at the leaf
    self._priority_sum[idx] = priority
    # Update tree, by traversing along ancestors. Continue until the root of the tree.
    while idx >= 2:
      # Get the index of the parent node
      idx //= 2
      # Value of the parent node is the sum of it's two children
      self._priority_sum[idx] = self._priority_sum[2 * idx] + self._priority_sum[2 * idx + 1]

  def _sum(self):
    # The root node keeps the sum of all values
    return self._priority_sum[1]

  def _min(self):
    # The root node keeps the minimum of all values
    return self._priority_min[1]

  def _find_prefix_sum_idx(self, prefix_sum):
    # find largest i such as p_1^alpha + p_2^alpha + ... p_i^alpha <= prefix_sum = P
    # Start from the root
    idx = 1
    while idx < self._capacity:
      # If the sum of the left branch is higher than required sum
      if self._priority_sum[idx * 2] > prefix_sum:
        # Go to left branch of the tree
        idx = 2 * idx
      else:
        # Otherwise go to right branch and reduce the sum of left branch from required sum
        prefix_sum -= self._priority_sum[idx * 2]
        idx = 2 * idx + 1
    # We are at the leaf node. Subtract the capacity by the index in the tree to get the index of actual value in the array
    return idx - self._capacity

Funzione che permette di riempire con `replay_memory_init_size` observations ricavate effettuando azioni random. Ogni episodio termina naturalmente o quando viene raggiunto un numero `episode_max_steps` di passi. 

In [16]:
def dqn_replay_memory_init(env, replay_memory, replay_memory_init_size, episode_max_steps):
  while True:
    state = env.reset()
    done = False
    step_count = 0
    print_step = math.ceil(replay_memory_init_size / 10)
    while step_count < episode_max_steps and not done:
      action = env.action_space.sample()
      new_state, reward, done, _ = env.step(action)
      replay_memory.append([state,action,reward,new_state,done])
      state = new_state
      step_count += 1
      replay_memory_len = len(replay_memory)
      if replay_memory_len % print_step:
        print(f'\r{replay_memory_len}/{replay_memory_init_size}', end='')
      if replay_memory_len >= replay_memory_init_size:
        print()
        return

## Action model update

In questa parte viene mostrato come la rete si aggiorna. 
L'aggiornamento della rete viene effettuato utilizzando un'implementazione chiamata Double DQN (https://arxiv.org/pdf/1509.06461.pdf) pubblicata da DeepMind nel 2015 per stabilizzare e migliorare l'addestramento. 

Il miglioramento si basa sul fatto che la rete _target_ tende a sovrastimare i Q-Value. Si supponga che tutte le azioni siano ugualmente buone: i Q-Values stimati dal modello target dovrebbero essere identici, ma trattandosi di approssimazioni, alcune potrebbero essere leggermente maggiori di altre, per puro caso. Il modello target selezionerà sempre il Q-Value più grande, che sarà leggermente maggiore del Q-Value medio, molto probabilmente sovrastimando il vero Q-Value. Per risolvere questo problema, hanno proposto di utilizzare il modello online (_action_model_) anziché il target quando si selezionano le azioni migliori per gli stati successivi e di utilizzare solo il modello di target per stimare i Q-Value per queste migliori azioni.

Il valore target diventa da:
$$Y^{DQN}_t \equiv R_{t+1} + \gamma \cdot \max_{a} Q(S_{t+1}, a; \theta_t^-) $$
a:
$$Y^{DoubleQ}_t \equiv R_{t+1} + \gamma \cdot Q(S_{t+1}, \underset{x}{\mathrm{argmax}}Q(S_{t+1}, a; \theta_t); \theta_t^-) $$


`update_action_model` è una funzione generica che astrae dalla tipologia di approccio utilizzato perche in ingresso prende i soliti parametri (`state_batch`, `action_batch`, `reward_batch`, `done_batch`, `gamma` e `dqn_action_model`), i `weights` (utilizzati nel caso si usi PER) e `target_new_state_q_values`, calcolati indipendentemente dall'algoritmo scelto.  

In [17]:
def update_action_model(dqn_action_model, 
                        target_new_state_q_values, 
                        gamma,
                        state_batch, 
                        action_batch, 
                        reward_batch, 
                        done_batch,
                        weights):

  # find the action model Q values for all possible actions given the current state batch
  predicted_state_q_values = dqn_action_model.predict(state_batch, verbose=False)
  predicted_Q_values_copy = predicted_state_q_values.copy()
  # estimate the target values y_i
  # for the action we took, use the target model Q values  
  # for other actions, use the action model Q values
  # in this way, loss function will be 0 for other actions
  for i,(a,r,new_state_q_values, done) in enumerate(zip(action_batch,reward_batch,target_new_state_q_values, done_batch)): 
      if not done:  
        target_value = r + gamma * np.amax(new_state_q_values)
      else:         
        target_value = r
      predicted_state_q_values[i][a] = target_value #y_i
  
  losses = dqn_action_model.loss(predicted_state_q_values, predicted_Q_values_copy)  
  # update weights of action model using the train_on_batch method 
  dqn_action_model.train_on_batch(state_batch, predicted_state_q_values, sample_weight=weights)
  
  return losses

Computa i *target_new_state_q_values* con DoubleDQN

In [18]:
def double_dqn_update(dqn_action_model, dqn_target_model, mini_batch, gamma, weights=None):
  # the transition mini-batch is divided into a mini-batch for each element of a transition
  state_batch,action_batch,reward_batch,new_state_batch,done_batch=mini_batch

  # pixel values are normalized in the range [0;1]
  norm_state_batch = state_batch/255.0
  norm_new_state_batch = new_state_batch/255.0

  # find the target model Q values for all possible actions given the new state batch
  next_Q_values = dqn_action_model.predict(norm_new_state_batch, verbose=0)  # not target.predict()
  best_next_actions = next_Q_values.argmax(axis=1) # select best action based on action model
  next_mask = tf.one_hot(best_next_actions, next_Q_values.shape[1]) # mask for best actions
  target_new_state_q_values = dqn_target_model.predict(norm_new_state_batch, verbose=0) * next_mask
  
  return update_action_model(dqn_action_model, 
                             target_new_state_q_values, 
                             gamma, 
                             norm_state_batch, 
                             action_batch, 
                             reward_batch, 
                             done_batch, 
                             weights)

### Altre funzioni di aggiornamento

Funzione che calcola i target Q_values in FixedDQN

In [20]:
def dqn_action_target_update(dqn_action_model, dqn_target_model, mini_batch, gamma, weights=None):
  # the transition mini-batch is divided into a mini-batch for each element of a transition
  state_batch,action_batch,reward_batch,new_state_batch,done_batch=mini_batch

  # pixel values are normalized in the range [0;1]
  norm_state_batch = state_batch/255.0
  norm_new_state_batch = new_state_batch/255.0

  # find the target model Q values for all possible actions given the new state batch
  target_new_state_q_values = dqn_target_model.predict(norm_new_state_batch, verbose=False)
  
  return update_action_model(dqn_action_model, 
                             target_new_state_q_values, 
                             gamma, 
                             norm_state_batch, 
                             action_batch, 
                             reward_batch, 
                             done_batch, 
                             weights)

Funzione che calcola i target Q-Values nella classica versione DQN

In [21]:
def dqn_update(dqn_action_model, loss_fn, mini_batch, gamma, weights=None):
  # the transition mini-batch is divided into a mini-batch for each element of a transition
  state_batch,action_batch,reward_batch,new_state_batch,done_batch=mini_batch

  # pixel values are normalized in the range [0;1]
  norm_state_batch = state_batch/255.0
  norm_new_state_batch = new_state_batch/255.0

  # find the target model Q values for all possible actions given the new state batch
  target_new_state_q_values = dqn_action_model.predict(norm_new_state_batch, verbose=False)
  
  return update_action_model(dqn_action_model, 
                             target_new_state_q_values,
                             gamma, 
                             norm_state_batch, 
                             action_batch, 
                             reward_batch, 
                             done_batch, 
                             weights)

## Training Function

Funzione di addestramento della rete. Addestra una rete DoubleDQN, ma basta effettuare qualche modifica per ottenere le altre.
- episode_count: numero di episodi con cui addestrare la rete
- env: l'environment
- dqn_action_model: l'online model
- dqn_target_model: l'offline model (potrebbe essere lo stesso `dqn_action_model`)
- episode_max_steps: numero di steps massimi per episodio,
- replay_memory_max_size: grandezza massima della reply memory,
- replay_memory_init_size: numero di transition con cui inizializzare la replay memory,
- alpha: quantità di priorità utilizzare,
- error_offset: offset sommato all'errore quando si aggiorna la PRM,
- batch_size: grandezza del mini-batch da campionare,
- step_per_update: ogni quanti step aggiornare l'online model,
- step_per_update_target_model: ogni quanti step aggiornare l'offline model,
- max_epsilon: $\epsilon$ massima,
- min_epsilon: $\epsilon$ minima,
- epsilon_decay: quanto far decadere $\epsilon$ in ogni step,
- gamma: il fattore di sconto $\gamma$,
- max_beta: $\beta$ massimo,
- min_beta: $\beta$ minimo,
- beta_increase: quanto incrementare $\beta$ ogni step,
- moving_avg_window_size: la finestra di reward con cui calcolare la media mobile,
- moving_avg_stop_thr: se `moving_avg_window_size` raggiunge questo valore il compito viene considerato completato e l'addestramento termina,
- load_folder: path dove vengono caricati i dati,
- save_folder: path dove salvare i dati,
- save_freq: ogni quanti episodi salvare il modello e i dati annessi

In [22]:
def dqn_training(episode_count,
                 env,
                 dqn_action_model,
                 dqn_target_model,
                 episode_max_steps=np.inf,
                 replay_memory_max_size=100_000,
                 replay_memory_init_size=20_000,
                 alpha=0.6,
                 error_offset=0.1,
                 batch_size=32,
                 step_per_update=4,
                 step_per_update_target_model=10_000,
                 max_epsilon=1,
                 min_epsilon=0.1,
                 epsilon_decay=1e-6,
                 gamma=0.99,
                 max_beta=1,
                 min_beta=0.4,
                 beta_increase=1e-6,
                 moving_avg_window_size=100,
                 moving_avg_stop_thr=None,
                 load_folder=None,
                 save_folder=None,
                 save_freq=50):
  
  data = load_model(load_folder) if load_folder else {}
  dqn_action_model = data.get('dqn_action_model', dqn_action_model)
  dqn_target_model = data.get('dqn_target_model', dqn_target_model)
  replay_memory = data.get('replay_memory', TreePrioritizedReplayMemory(replay_memory_max_size, alpha, error_offset))
    
  # 1. replay memory initial population
  if 0 <= len(replay_memory) < replay_memory_init_size:
    print('Replay memory initial population')
    dqn_replay_memory_init(env, replay_memory, replay_memory_init_size, episode_max_steps)

  # inizialize epsilon equal to max_epsilon
  epsilon = data.get('epsilon',max_epsilon)
  # initialize beta equal to min_beta
  beta = data.get('beta', min_beta)
  train_rewards = data.get('train_rewards',[])
  train_step_count = len(train_rewards) #T

  print('start training')
  for n in range(train_step_count, episode_count): 
    # for visualization purposes
    episode_start_time = time.time()    
    episode_start_epsilon = epsilon
    episode_start_beta = beta
    # episode init
    state = env.reset()
    episode_reward = 0
    step_count = 0  #t
    done = False

    while step_count < episode_max_steps and not done: 
      # decide whether to pick a random action or to exploit the already computed Q-values
      if random.uniform(0, 1) <= epsilon:
        action = env.action_space.sample()
      else:
        # 4. stacked frame normalization
        norm_state = state/255.0
        q_values = dqn_action_model.predict(norm_state[np.newaxis], verbose=False)
        action = np.argmax(q_values)
            
      # take the action and observe the outcome state and reward
      new_state, reward, done, _ = env.step(action)
            
      # store transition in the replay memory
      replay_memory.append([state, action, reward, new_state, done])
            
      # update the current state
      state = new_state
                        
      if train_step_count % step_per_update == 0 and len(replay_memory) >= batch_size:
        # get a random mini-batch from the replay memory
        *mini_batch, weights, indexes = replay_memory.sample_minibatch(batch_size, beta)
        # 7. update the action model weights
        losses = double_dqn_update(dqn_action_model, dqn_target_model, mini_batch, gamma, weights)
        replay_memory.update_priorities(indexes, losses)
                
      if train_step_count % step_per_update_target_model ==0:
          # copy weights from action to target model
          dqn_target_model.set_weights(dqn_action_model.get_weights())

      # reduce epsilon
      epsilon = max(min_epsilon, epsilon - epsilon_decay)
      # increase beta
      beta = min(max_beta, beta + beta_increase)
            
      # increase episode step count and total step count
      step_count += 1
      train_step_count += 1

      # add the current reward to the episode total reward
      episode_reward += reward 
        
    # put the episode total reward into a list for visualization purposes
    train_rewards.append(episode_reward)

    if n > 0 and n % save_freq == 0 and save_folder is not None:
      save_model(save_folder, dqn_action_model, dqn_target_model, train_rewards, epsilon, beta, replay_memory)

    # for visualization purposes
    episode_finish_time=time.time()
    episode_elapsed_time=episode_finish_time-episode_start_time
    episode_avg_step_time=episode_elapsed_time/step_count
    moving_avg_reward=statistics.mean(train_rewards[-moving_avg_window_size:])

    print_episode_info(n,
                       step_count,
                       train_step_count,
                       episode_start_epsilon,
                       epsilon,
                       episode_start_beta,
                       beta,
                       episode_elapsed_time,
                       episode_avg_step_time,
                       episode_reward,
                       moving_avg_reward)        
    # condition to consider the task solved
    if moving_avg_stop_thr is not None and moving_avg_reward > moving_avg_stop_thr:
      break

  # return a list containing the total rewards of all training episodes
  return train_rewards

## Training

### Iperparametri

In [28]:
episode_count = 10              # Total number of training episodes
episode_max_steps= 10_000            # Maximum number of steps per episode
save_freq = 50                      # model save frequency  
load_folder = save_folder = 'DQN_pong' # folder to save data
top_crop = 30                       # y coordinate of the top of the region of interest
bottom_crop = 195                   # y coordinate of the bottom of the region of interest
left_crop = 5                       # x coordinate of the left of the region of interest
right_crop = 155                    # x coordinate of the right of the region of interest

crops = (top_crop, right_crop, bottom_crop, left_crop)
shape = (84, 84)

frame_skip = 4  # how many frames skip
stacked_frames = 4 # how many frames stack together 

replay_memory_max_size = 100_000    # The maximum number of transitions stored into the replay memory. The Deepmind paper suggests 1M however this may cause memory issues.
replay_memory_init_size = 20_000    # The maximum number of transitions stored into the replay memory. The Deepmind paper suggests 50K.
batch_size = 32                     # The mini-batch size

step_per_update = 4                 # The number of total steps executed between successive updates of the action model weights
step_per_update_target_model = 10_000  # The number of total steps executed between successive replaces of the target model weights

max_epsilon = 1.0                     # Exploration probability at start
min_epsilon = 0.1                     # Minimum exploration probability
epsilon_decay = (max_epsilon-min_epsilon) / 1_000_000.0  # Decay for exploration probability

max_beta = 1  # how much compensate the bias at the end
min_beta = 0.4 # how much compensate the bias at the start
beta_increase = (max_epsilon-min_epsilon) / 1_000_000.0 # increase for beta

alpha = 0.6 # how much weight 
error_offset = 0.1

gamma = 0.99                        # Discount factor
moving_avg_window_size = 100          # Number of consecutive episodes to be considered in the calculation of the total reward moving average
moving_avg_stop_thr = 21              # Minimum value of the total reward moving average to consider the task solved
action_repeat_probability = 0.25    # probability to repeat last action instead the action
learning_rate = 2.5e-4              

Make the env

In [29]:
env = make_env(env_name, stacked_frames, frame_skip, action_repeat_probability, shape, crops)

create and compile the model

In [30]:
# To train dueling use build_dueling_dqn
dqn_action_model=build_dueling_dqn(action_count=env.action_space.n)
dqn_target_model=build_dueling_dqn(action_count=env.action_space.n)
dqn_target_model.set_weights(dqn_action_model.get_weights())

loss = keras.losses.Huber(reduction='none')
optimizer = keras.optimizers.legacy.Adam(learning_rate = learning_rate, clipnorm = 1.0)
dqn_action_model.compile(optimizer = optimizer, loss = loss)  

In [31]:
keras.utils.plot_model(dqn_action_model, to_file='model_strucure.png', show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


### Train the model

In [32]:
train_rewards = dqn_training(episode_count,
                             env,
                             dqn_action_model,
                             dqn_target_model,
                             episode_max_steps,
                             replay_memory_max_size,
                             replay_memory_init_size,
                             alpha,
                             error_offset,
                             batch_size,
                             step_per_update,
                             step_per_update_target_model,
                             max_epsilon,
                             min_epsilon,
                             epsilon_decay,
                             gamma,
                             max_beta,
                             min_beta,
                             beta_increase,
                             moving_avg_window_size,
                             moving_avg_stop_thr,
                             load_folder,
                             save_folder,
                             save_freq)

ERROR: No file or directory found at DQN_pong/dqn_action_model
Data not loaded
Replay memory initial population
19999/20000
start training


2023-04-03 14:15:19.217751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2023-04-03 14:15:19.538841: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Episode: 0 Steps: 842[842] Epsilon: [1.000->0.999] Beta: [0.400->0.401] Time: 45.7s[0.05s] Total reward: -20.0[-20.0]
Episode: 1 Steps: 825[1667] Epsilon: [0.999->0.998] Beta: [0.401->0.402] Time: 41.9s[0.05s] Total reward: -21.0[-20.5]
Episode: 2 Steps: 1066[2733] Epsilon: [0.998->0.998] Beta: [0.402->0.402] Time: 54.9s[0.05s] Total reward: -18.0[-19.7]
Episode: 3 Steps: 825[3558] Epsilon: [0.998->0.997] Beta: [0.402->0.403] Time: 42.6s[0.05s] Total reward: -21.0[-20.0]
Episode: 4 Steps: 793[4351] Epsilon: [0.997->0.996] Beta: [0.403->0.404] Time: 40.5s[0.05s] Total reward: -21.0[-20.2]
Episode: 5 Steps: 872[5223] Epsilon: [0.996->0.995] Beta: [0.404->0.405] Time: 44.9s[0.05s] Total reward: -21.0[-20.3]
Episode: 6 Steps: 765[5988] Epsilon: [0.995->0.995] Beta: [0.405->0.405] Time: 39.4s[0.05s] Total reward: -21.0[-20.4]
Episode: 7 Steps: 871[6859] Epsilon: [0.995->0.994] Beta: [0.405->0.406] Time: 44.8s[0.05s] Total reward: -20.0[-20.4]
Episode: 8 Steps: 978[7837] Epsilon: [0.994->0.9

In [ ]:
#!zip -r dqn_train.zip DQN_breakout

# Stable Baselines 3

## Introduzione agli algoritmi **Policy-Based**

Nei metodi **policy-based** invece di apprendere una *value function* che calcola il _valore atteso_ della somma dei rewards dato uno stato ed un'azione si apprende direttamente la policy che mappa da stato ad azione. 
Questo significa che si ottimizza direttamente la policy $\pi$.

La policy potrebbe essere deterministica o stocastica.
In questo progetto si tratterà solamente la versione stocastica.
Una policy stocastica restituisce una distribuzione di probabilità sulle azioni:

<p><img src="https://cdn-media-1.freecodecamp.org/images/1*YCABimP7x1wZZZKqz2CoyQ.png" width="400"/></p>

Questo significa che ogni possibile azione ha una certa probabilità per essere effettuata. Questo viene definito **Partially Observable Markov Decision Process**.

I modelli policy-based hanno diversi vantaggi:

- hanno migliori proprietà di convergenza. Rispetto ai metodi _value-based_, in cui ci sono oscillazioni dovute alla stima degli _action values_, i policy-based ottimizzano i parametri utilizzando il gradiente per effettuare aggiornamenti piu "delicati" ad ogni step. Dato che si segue il gradiente si ha la certezza di convergere in un minimo (sia locale che globale). Inoltre è più facile da approssimare che una value function (e.g. in Pong è difficile assegnare uno score al movimento della racchetta)

<p><img src="https://cdn-media-1.freecodecamp.org/images/1*0lYcY5TBSqfNwdu8TduB6g.png" width="400"/></p>

- sono più efficaci in spazi dimensionali molto grandi o quando si usano azioni continue. Con DQN si calcola un valore per ogni possibile azione, ma se le azioni fossero infinite o molte non sarebbe possibile. I metodi policy-based trovano i valori ottimi in modo iterativo ottimizzando i parametri.
- stocastic policy, mentre _value-based_ non possono. Questo comporta che
  -  non è necessario implementare un sistema di _exploration/exploitation_.
  -  Elimina il _perceptual aliasing_, un fenomeno per il quale due stati sembrano gli stessi ma necessitano di azioni differenti. In questi casi un metodo _value-based_ pensando che i due stati siano uguali sceglierebbe, in uno dei due casi, sempre l'azione sbagliata in modo deterministico (questo effetto si riduce con la _epsilon greedy strategy_, però nelle fasi avanzate dell'addestramento perde la sua efficacia). La conseguenza è che l'agente potrebbe spendere un sacco di tempo per continuare correttamente. Una policy stocastica ottimale effettuerebbe un'azione in modo random cosi che l'agente raggiunga l'obbiettivo con una probabilità più alta.
  - efficace nei casi in cui un approccio deterministico non sarebbe ottimo (e.g. sasso-carta-forbici)
I metodi policy-based hanno un grosso svantaggio: spesso convergono in un minimo locale invece del massimo globale. Diversamente da DQN, che cerca sempre il massimo, questi metodi convergono step by step lentamente, il che potrebbe portare a tempi di addestramenti più lunghi. 

## Policy Gradient Theorem
Sia $x$ l'azione che si andrà ad intraprendere, $s$ lo stato attuale e $\theta$ i pesi della rete. La rete darà in output delle probabilità $p(x \mid s; \theta)$ che dipendono dallo stato $s$ e dai pesi $\theta$. Si vuole quindi massimizzare la _reward attesa_ $r(x)$. $r(x)$ dipende dall'azione $x$ che si sceglie.

$$ J(\theta) = E_{x \sim p(x \mid s; \theta)}[r(x)]$$

Per massimizzare la funzione si effettua il _gradient ascent_. Per farlo occorre $\nabla_\theta J(\theta)$.

<p><img src="https://www.sitolorenzogardini.altervista.org/images/expectation_gradient.PNG" width="600"/></p>

**Da questo risulta che è il valore atteso della reward moltiplicato per il logaritmo delle probabilità delle azioni**

Questo vale però per una singola azione. Ora si considerano delle traiettorie, una sequenza di azioni.

### Traiettorie

Si assume il principio della _Markov Property_: lo stato successivo $s'$ dipende dallo stato corrente $s$ e dall'azione $a$ ma dati $(s,a)$ è indipendente da tutti gli stati e le azioni precedenti.
La probabilità di una traiettoria è definita come:

<p><img src="https://www.sitolorenzogardini.altervista.org/images/transition_prob.PNG" width="500"/></p>

Considerando un processo di generazione stocastica che genera delle traiettorie $(s, a, r)$:

$$\tau = (s_0, a_0, r_0, s_1, a_1, r_1,...,s_{T-1}, a_{T-1}, r_{T-1}, s_T)$$

Si vuole calcolare il valore atteso delle rewards su una traiettoria:

<p><img src="https://www.sitolorenzogardini.altervista.org/images/expected_reward_trajectory_unfolded.PNG" width="400"/></p>

dove $R(\tau)$ è semplicemente la somma delle rewards della traiettoria.

$$R(\tau) = \sum\limits_{t=0}^{T-1}r_t$$

### Probabilità di una traiettoria data una policy

<p><img src="https://www.sitolorenzogardini.altervista.org/images/probability_of_trajectory_given_policy.PNG
" width="600"/></p>

### Massimizzare il valore eatteso del ritorno di una traiettoria

Dato che 

$$\log p(\tau \mid \theta) = log \;\mu(s_0) +  \sum\limits_{t=0}^{T-1} \Big[ log \; \pi (a_t \mid s_t, \theta) + log \; p(s_{t+1},r_t \mid s_t, a_t) \Big]$$

il $\nabla_\theta \log p(\tau \mid \theta)$, diventa :

$$\nabla_{\theta} \sum\limits_{t=0}^{T-1} log \; \pi (a_t \mid s_t, \theta)$$

dato che è l'unico termine che dipende da $\theta$. Quindi 

$$\nabla_{\theta} \mathbb{E}_{\tau} [R(\tau)] = \mathbb{E}_{\tau} \Big[ R(\tau) \nabla_{\theta} \sum\limits_{t=0}^{T-1} log \; \pi (a_t \mid s_t, \theta) \Big]$$


Da notare che non dipende dalla distribuzione di probabilità dell'environment (quello che sopra era riportato come transition function, $p(s_{t+1},r_t \mid s_t, a_t)$). Per questo i modelli Gradient policy vengono chiamati **Model Free Models**.

### Policy gradient ascent (esempio con REINFORCE)

L'aggiornamento dei pesi della rete viene effettuato in modo iterativo in questo modo:

$$ \theta \leftarrow \theta + \alpha \gamma^t G \nabla_\theta \; ln \; \pi (a_t \mid s_t, \theta)$$

dove $G$ è il return dello step $t$

## A2C

Formula del gradiente della policy:

<p><img src="https://miro.medium.com/max/1100/1*YQqZyAJ1QehPXFW36TKwmw.webp" width='300'/></p>

Questo approccio ha diversi problemi: _noisy gradient_ e una variabilità molto alta. Questo è dato dal fatto che i _samples_ possono avere molta variabilità tra loro e quindi anche tra le rewards. Questo causa _noisy gradient_ ed un conseguente apprendimento instabile. Inoltre le rewards possono essere **sparse**, la maggior parte delle azioni ha come reward 0. Questo causa un rallentamento nell'addestramento.

## Baseline
Per introdurre stabilità si utilizza una baseline:
<p><img src="https://miro.medium.com/max/1100/1*45Wfsmp3KnzbJPKWt7GXpg.webp" width='400'/></p>

Questo produce gradienti più piccoli e quindi aggiornamenti più piccoli e più stabili.

Questo verrà combinato successivamente.

## Actor-Critic Method

Si può decomporre $G_t$ come:

<p><img src="https://miro.medium.com/max/1100/1*p0R0jWEaUAk2CEo-rk7MrQ.webp" width='500'/></p>

Il secondo valore atteso combacia con il _Q value_:
<p><img src="https://miro.medium.com/max/1100/1*1CcRr_bsaebzGXyi6gC6yA.webp" width='300'/></p>

I grandiente diventa:
<p><img src="https://miro.medium.com/max/1100/1*JYp-uQrMJKEHadx4RBrR1A.webp" width='500'/></p>

dove $Q_w$ è una rete con pesi $w$.

Questo porta al metodo _**Actor-Critic**_:

- Il _Critic_ stima la _value function_. Questo potrebbe essere il valore dell'azione (il valore _Q_ ) o il valore dello stato (il valore _V_ ).
- L' _Actor_ aggiorna la distribuzione della policy nella direzione suggerita dal _Critic_.

Sia _Critic_ che _Actor_ sono delle reti neruali, nella formula di prima il _Critic_ è dato dalla funzione _Q_, quindi viene chiamato _**Q Actor-Critic**_.

Algoritmo:
<p><img src="https://miro.medium.com/max/1100/1*BVh9xq3VYEsgz6eNB3F6cA.webp" width='500'/></p>

## Advantage Actor Critic

Come baselines $b(s_t)$ si utilizza la value function $V(s_t)$

<p><img src="https://miro.medium.com/max/1100/1*GjirmHTNdxHgo1Z8iQjDbg.webp" width='300'/></p>

Questo si può interpretare come quanto sia migliore intraprendere l'azione $a$ rispetto a quella scelta dalla policy $\pi$.

_Q_ si può scrivere rispetto a _V_.
<p><img src="https://miro.medium.com/max/1100/1*ffw4Ri8eyB5FUArQq9wZeg.webp" width='300'/></p>

<p><img src="https://miro.medium.com/max/1100/1*pgGvGTyJ7gtlCICD5M-Z-A.webp" width='400'/></p>

Il gradiente diventa:
<p><img src="https://miro.medium.com/max/1100/1*pjE0o_wWTdcjprdDQFnwog.webp" width='400'/></p>

## Implementazione

Introdotta la versione A3C da Google nel 2016.
https://arxiv.org/pdf/1602.01783.pdf

In sostanza, A3C implementa la formazione parallela in cui più lavoratori in ambienti paralleli aggiornano in modo indipendente una funzione di valore globale , quindi "asincrona". Uno dei principali vantaggi di avere attori asincroni è l'esplorazione efficace ed efficiente dello spazio degli stati.

<p><img src="https://miro.medium.com/max/1100/1*EiFOXH3MZ0r_a_aKLd9r8A.webp" width='400'/></p>

## A2C vs A3C
A2C funziona come A3C ma senza la parte asincrona. Questo significa che è presente un coordinatore per i vari agenti. È stato dimostrato empiricamente che A2C ha performance comparabili, se non migliori, della controparte asincrona. 
https://openai.com/blog/baselines-acktr-a2c/.

Da qui si evidenzia il fatto che non ci siano miglioramenti nella versione asyncrona. _"This A2C implementation is more cost-effective than A3C when using single-GPU machines, and is faster than a CPU-only A3C implementation when using larger policies"_.

<p><img src="https://www.researchgate.net/publication/344581679/figure/fig1/AS:945026666336258@1602323323378/Asynchronous-learning-with-A2C-and-A3C-algorithms.ppm" width='400'/></p>



## PPO

https://arxiv.org/pdf/1707.06347v2.pdf

L'idea con Proximal Policy Optimization (PPO) è che si vuole migliorare la stabilità dell'addestramento della policy limitando le modifiche apportate alla policy in ogni epoca di training: si vuole evitare di effettuare aggiornamenti troppo grandi. Questo per due motivi:

- aggiornamenti più piccoli della policy hanno maggiori probabilità di convergere in una soluzione ottima

- un aggiornamento troppo grande può comportare quello che viene chiamato 'falling off the cliff' (caduta dal precipizio) in cui la policy impiega molto tempo per recuperare oppure diverge completamente.

<p><img src='https://huggingface.co/blog/assets/93_deep_rl_ppo/cliff.jpg' width='200'/></p>

In PPO la policy viene aggiornata in modo conservativo. Per farlo occorre misurare quanto un aggiornamento si è discostato da quello precedente. Questo valore viene poi _clippato_ in un range $[1 - \epsilon, 1 + \epsilon]$ in modo che la policy attuale non si discosti troppo da quella vecchia (da qui il termine _proximal policy_).

## Clipped Surrogate Objective

La funzione obbiettivo della policy vanilla ha il problema di essere lenta a convergere ed è troppo variabile in fase di addestramento.

Si introduce una nuova funzione che introduce il _clipping_ per evitare aggiornamenti troppo repentini della policy.
<p><img src='https://www.sitolorenzogardini.altervista.org/images/clipped_surrogate_objective.PNG' width='700'/></p>

### Ratio $\rho_t(\theta)$
Il _ratio_ corrisponde alla probabilità di scegliere l'azione $a_t$ nello stato $s_t$ nella policy attuale diviso da quella precedente:
<p><img src='https://www.sitolorenzogardini.altervista.org/images/ratio.PNG' width='300'/></p>

- se $r_t(\theta) > 1$ l'azione $a_t$ nello stato $s_t$ è più probabile nella policy corrente che in quella vecchia
- se $r_t(\theta) \in [0, 1]$ l'azione è meno probabile nella policy corrente che in quella vecchia.

Questo permette di stimare la divergenza tra la policy vecchia e quella nuova.

Questo rapporto sistituisce il $\log \pi_\theta(a_t | s_t)$ della funzione obbiettivo base. 

<p><img src='https://miro.medium.com/max/1476/0*S949lemw0fEDVPJE' width='400'/></p>

Questo però non basta, in quanto se un'azione presa è molto più probabile nella policy corrente causerebbe un aggiornamento eccessivo della policy. 

## Clipped Surrogate Objective function

Si vincola quindi la funzione obbiettivo in modo che il _ratio_ non si scosti di molto da 1 (nel paper $r_t (\theta) \in [0.8, 1.2]$). In questo modo ci si assicura che non ci siano aggiornamenti della policy troppo grandi perchè la nuova policy non può essere troppo diversa da quella vecchia.

<p><img src='https://miro.medium.com/max/695/1*rSpNhA8WkB0QqefM3vBYtA.png' width='400'/></p>

Si configurano questi casi:

<p><img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*8kQ1WK6jgJcZw8HdPwzKDg.png' width='500'/></p>

Quando la funzione obiettivo viene clippata il gradiente diventa 0 (dato che $1 - \epsilon$ non dipende da $\theta$). Cosi facendo vengono scoraggiate tutti gli aggiornamenti o troppo grandi.



In [34]:
import gym
#import imageio
import numpy as np
from pandas import Series
from gym import spaces
import cv2

# models
from stable_baselines3.a2c import A2C
from stable_baselines3.ppo import PPO
from stable_baselines3 import DQN
# stable wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecMonitor
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike

## Utility

In [35]:
class StickyActionEnv(gym.Wrapper):
  def __init__(self, env: gym.Env, action_repeat_probability: float) -> None:
    super().__init__(env)
    self.action_repeat_probability = action_repeat_probability

  def reset(self, **kwargs):
    self._sticky_action = 0  # NOOP
    return self.env.reset(**kwargs)

  def step(self, action: int):
    if self.np_random.random() >= self.action_repeat_probability:
      self._sticky_action = action
    return self.env.step(self._sticky_action)

class SkipEnv(gym.Wrapper):
  def __init__(self, env: gym.Env, skip: int = 4) -> None:
    super().__init__(env)
    self._skip = skip

  def step(self, action: int):
    total_reward = 0.0
    done = False
    for _ in range(self._skip):
      obs, reward, done, info = self.env.step(action)
      total_reward += reward
      if done:
          break
    return obs, total_reward, done, info
  
class PreprocessImageEnv(gym.Wrapper):
  def __init__(self, env: gym.Env, screen_size=84, crops=(None, None, None, None)) -> None:
    super().__init__(env)
    self.screen_size = screen_size
    self.crops = crops
    self.observation_space = spaces.Box(
        low=0, 
        high=255, 
        shape=(screen_size, screen_size, 1), 
        dtype=env.observation_space.dtype)

  def step(self, action):
    obs, reward, game_over, info = self.env.step(action)
    return self._process(obs), reward, game_over, info

  def reset(self):
    return self._process(self.env.reset())

  def _process(self, obs):
    gray = cv2.cvtColor(obs, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape

    top_crop, right_crop, bottom_crop, left_crop = self.crops
    top_crop = 0 if top_crop is None else top_crop
    right_crop = width if right_crop is None else right_crop
    bottom_crop = height if bottom_crop is None else bottom_crop
    left_crop = 0 if left_crop is None else left_crop

    cut_obs = gray[top_crop:bottom_crop, left_crop:right_crop]
    resized = cv2.resize(cut_obs, (self.screen_size, self.screen_size), interpolation=cv2.INTER_NEAREST)
    int_image = np.asarray(resized, dtype=np.uint8)
    return np.expand_dims(int_image, axis=2)

In [36]:
def create_training_env(env_name, 
                 n_envs=1,
                 action_repeat_probability=0.25, 
                 stack_frames=4, 
                 frame_skip=4,
                 crops = [None, None, None, None],
                 monitor_path=None, 
                 difficulty=1):
  
  def make_env(env_name, index=0):
    env = gym.make(env_name, difficulty=difficulty)
    env = StickyActionEnv(env, action_repeat_probability)
    env = SkipEnv(env, frame_skip)
    env = PreprocessImageEnv(env, crops=crops)
    return env

  vec_env = DummyVecEnv([lambda: make_env(env_name, i) for i in range(n_envs)])
  stacked_vec_env = VecFrameStack(vec_env,  n_stack=stack_frames)
  return VecMonitor(stacked_vec_env, monitor_path)


def make_gif(model, gif_name):
  images = []
  obs = model.env.reset()
  img = model.env.render(mode="rgb_array")
  while(True):
    images.append(img)
    action, _ = model.predict(obs)
    obs, _, done, _ = model.env.step(action)
    img = model.env.render(mode="rgb_array")
    if done:
      break
  imageio.mimsave(f"{gif_name}.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

Iperparametri ottimizzati per ciascun algoritmo

In [45]:
time_stamps = 100_000
env_name = 'PongNoFrameskip-v4'
stack_frames = 4
difficulty = 3
action_repeat_probability = -1
crops = [34, None, 194, None]
frame_skip = 4

# ppo_max_env = 8
# a2c_max_env = 16
# dqn_max_env = 16

dqn_hyperparams = {
  'buffer_size': 100_000,
  'learning_rate': 1e-4,
  'batch_size': 32,
  'learning_starts': 100_000,
  'target_update_interval': 1000,
  'train_freq': 4,
  'gradient_steps': 1,
  'exploration_fraction': 0.1,
  'exploration_final_eps': 0.01,
}

ppo_hyperparams = {
  'n_steps': 128,
  'n_epochs': 4,
  'batch_size': 256,
  'learning_rate': 2.5e-4,
  'clip_range': 0.1,
  'vf_coef': 0.5,
  'ent_coef': 0.01,
}

a2c_hyperparams = {
  'ent_coef': 0.01,
  'vf_coef': 0.25,
  'policy_kwargs': dict(optimizer_class=RMSpropTFLike, optimizer_kwargs=dict(eps=1e-5))
}

algo = DQN
n_envs = 16
hyper_params = dqn_hyperparams
save_path = f'{algo.__name__}/{algo.__name__}_{n_envs}'

creazione dell'env

In [46]:
training_env = create_training_env(env_name, 
                   n_envs, 
                   action_repeat_probability, 
                   stack_frames, 
                   frame_skip, 
                   crops,
                   monitor_path=save_path, 
                   difficulty=difficulty)

Addestramento dell'algoritmo selezionato

In [47]:
model = algo('CnnPolicy', training_env, **hyper_params, verbose=True)
model.learn(time_stamps, log_interval=1)
#model.save(save_path)

Using cuda device
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 765      |
|    ep_rew_mean      | -21.0    |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 1        |
|    fps              | 1199     |
|    time_elapsed     | 10       |
|    total_timesteps  | 12240    |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 765      |
|    ep_rew_mean     | -21.0    |
| time/              |          |
|    episodes        | 2        |
|    fps             | 1199     |
|    time_elapsed    | 10       |
|    total_timesteps | 12240    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 765      |
|    ep_rew_mean     | -21.0    |
| time/              |          |
|    episodes        | 3        |
|    fps    

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 864      |
|    ep_rew_mean     | -20.5    |
| time/              |          |
|    episodes        | 23       |
|    fps             | 1197     |
|    time_elapsed    | 22       |
|    total_timesteps | 27152    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 864      |
|    ep_rew_mean     | -20.5    |
| time/              |          |
|    episodes        | 24       |
|    fps             | 1197     |
|    time_elapsed    | 22       |
|    total_timesteps | 27152    |
---------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 863      |
|    ep_rew_mean      | -20.52   |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 25       |
|    fps              | 1197     |
|    time_elapsed     | 22       |
|    

------------------------------------
| rollout/            |            |
|    ep_len_mean      | 856        |
|    ep_rew_mean      | -20.545454 |
|    exploration_rate | 0.01       |
| time/               |            |
|    episodes         | 44         |
|    fps              | 1198       |
|    time_elapsed     | 35         |
|    total_timesteps  | 42480      |
------------------------------------
------------------------------------
| rollout/            |            |
|    ep_len_mean      | 858        |
|    ep_rew_mean      | -20.533333 |
|    exploration_rate | 0.01       |
| time/               |            |
|    episodes         | 45         |
|    fps              | 1197       |
|    time_elapsed     | 35         |
|    total_timesteps  | 42544      |
------------------------------------
------------------------------------
| rollout/            |            |
|    ep_len_mean      | 858        |
|    ep_rew_mean      | -20.543478 |
|    exploration_rate | 0.01       |
|

------------------------------------
| rollout/            |            |
|    ep_len_mean      | 868        |
|    ep_rew_mean      | -20.446154 |
|    exploration_rate | 0.01       |
| time/               |            |
|    episodes         | 65         |
|    fps              | 1198       |
|    time_elapsed     | 52         |
|    total_timesteps  | 62720      |
------------------------------------
------------------------------------
| rollout/            |            |
|    ep_len_mean      | 868        |
|    ep_rew_mean      | -20.454546 |
|    exploration_rate | 0.01       |
| time/               |            |
|    episodes         | 66         |
|    fps              | 1199       |
|    time_elapsed     | 55         |
|    total_timesteps  | 66400      |
------------------------------------
------------------------------------
| rollout/            |            |
|    ep_len_mean      | 867        |
|    ep_rew_mean      | -20.447762 |
|    exploration_rate | 0.01       |
|

------------------------------------
| rollout/            |            |
|    ep_len_mean      | 864        |
|    ep_rew_mean      | -20.476744 |
|    exploration_rate | 0.01       |
| time/               |            |
|    episodes         | 86         |
|    fps              | 1199       |
|    time_elapsed     | 68         |
|    total_timesteps  | 82272      |
------------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 862       |
|    ep_rew_mean      | -20.48276 |
|    exploration_rate | 0.01      |
| time/               |           |
|    episodes         | 87        |
|    fps              | 1199      |
|    time_elapsed     | 68        |
|    total_timesteps  | 82304     |
-----------------------------------
------------------------------------
| rollout/            |            |
|    ep_len_mean      | 862        |
|    ep_rew_mean      | -20.477272 |
|    exploration_rate | 0.01       |
| time/     

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 870      |
|    ep_rew_mean      | -20.38   |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 107      |
|    fps              | 1199     |
|    time_elapsed     | 81       |
|    total_timesteps  | 97856    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 870      |
|    ep_rew_mean      | -20.38   |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 1198     |
|    time_elapsed     | 82       |
|    total_timesteps  | 98432    |
----------------------------------


Create a gif with the model

In [ ]:
# make_gif(model, f'{algo.__name__}_{n_envs}')

Gif con il risultato dell'addestramento con DQN e 1 environment

<p><img src='https://www.sitolorenzogardini.altervista.org/images/DQN_gif.gif' width='300'/></p>

# Tensorflow Agents

Tensorflow Agents permette di creare un ambiente completo di reinforcement learning in modo modulare 

<p><img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*pJ5utPNBZFcJRhdxX-Fsfw.png' width='500'/></p>

Installazione e import di librerie base

In [48]:
import cv2
from collections import deque
import gym
from gym import spaces
import numpy as np
from pandas import Series
import os

## Log

In [49]:
from tf_agents.eval.metric_utils import log_metrics
import logging
logging.getLogger().setLevel(logging.INFO)

## Environment Wrappers and Atari Preprocessing

I wrappers utilizzati presentati nella parte manuale 

In [50]:
class SkipEnv(gym.Wrapper):
  def __init__(self, env: gym.Env, skip: int = 4) -> None:
    super().__init__(env)
    self._skip = skip

  def step(self, action: int):
    total_reward = 0.0
    done = False
    for _ in range(self._skip):
      obs, reward, done, info = self.env.step(action)
      total_reward += reward
      if done:
          break
    return obs, total_reward, done, info

class TFWrapper(gym.Wrapper):
  def __init__(self, env: gym.Env):
    super().__init__(env)
    self.game_over = False

  def reset(self):
    self.game_over = False
    return self.env.reset()

  def step(self, action):
    obs, reward, done, info = self.env.step(action)
    if done:
      self.game_over = True
    return obs, reward, done, info
  
class PreprocessImageEnv(gym.Wrapper):
  def __init__(self, env: gym.Env, screen_size=84, crops=(None, None, None, None)) -> None:
    super().__init__(env)
    self.screen_size = screen_size
    self.crops = crops
    self.observation_space = spaces.Box(
        low=0, 
        high=255, 
        shape=(screen_size, screen_size, 1), 
        dtype=env.observation_space.dtype)

  def step(self, action):
    obs, reward, game_over, info = self.env.step(action)
    return self._process(obs), reward, game_over, info

  def reset(self):
    return self._process(self.env.reset())

  def _process(self, obs):
    gray = cv2.cvtColor(obs, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape

    top_crop, right_crop, bottom_crop, left_crop = self.crops
    top_crop = 0 if top_crop is None else top_crop
    right_crop = width if right_crop is None else right_crop
    bottom_crop = height if bottom_crop is None else bottom_crop
    left_crop = 0 if left_crop is None else left_crop

    cut_obs = gray[top_crop:bottom_crop, left_crop:right_crop]
    resized = cv2.resize(cut_obs, (self.screen_size, self.screen_size), interpolation=cv2.INTER_NEAREST)
    int_image = np.asarray(resized, dtype=np.uint8)
    return np.expand_dims(int_image, axis=2)

Creazione dell'env. Quest'ultimo deve essere wrappato con `TFPyEnvironment` in modo che possa essere ottimizzato con TF.

<p><img src='https://www.sitolorenzogardini.altervista.org/images/env_creation.png' width='400'/></p>

In [51]:
from tf_agents.environments import suite_atari
from tf_agents.environments.atari_preprocessing import AtariPreprocessing
from tf_agents.environments.atari_wrappers import FrameStack4
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from functools import partial
from tf_agents.environments import atari_wrappers
from tf_agents.environments import suite_gym 
from tf_agents.environments.suite_gym import wrap_env
from stable_baselines3.common.monitor import Monitor

stack_frames = 4
difficulty = 3
action_repeat_probability = 0.25
crops = [34, None, 194, None]
frame_skip = 4
seed = 1234
screen_size = 84
max_episode_steps = 25_000 # <=> 100k ALE frames since 1 step = 4 frames
env_name = 'PongNoFrameskip-v4'
dueling = True
double = True
save_path = 'tf_' + ('DDQN' if double else 'DQN') + '/' + ('dueling_' if dueling else '') + 'rewards.csv'


def wrap(gym_env, wrappers, max_episode_steps):
  wrapped = suite_gym.wrap_env(
      gym_env,
      max_episode_steps=max_episode_steps,
      gym_env_wrappers=wrappers,
      time_limit_wrapper=atari_wrappers.AtariTimeLimit,
      spec_dtype_map = {gym.spaces.Box: np.uint8},
      auto_reset=False)
  return TFPyEnvironment(wrapped)

env = gym.make(env_name, difficulty=difficulty)
env.seed(seed)

tf_training_env = wrap(env, 
                    [TFWrapper,
                     partial(SkipEnv, skip=frame_skip),
                     partial(PreprocessImageEnv, crops=crops),
                     FrameStack4,
                     partial(Monitor, filename=save_path)],
                    max_episode_steps)

## Q-Net

Qui vengono definite le due versioni della rete, una classica DQN convoluzionale (creata facilmente utilizzando la classe `QNetwork`) mentre la seconda Dueling (questa viene wrappata da un layer `Sequential` per renderla compatibile con TF agents).
<p><img src='https://www.sitolorenzogardini.altervista.org/images/network_creation.png' width='400'/></p>

In [52]:
from tf_agents.networks.q_network import QNetwork
from tensorflow import keras
import tensorflow as tf 
import numpy as np
from keras.layers import Input, Conv2D, Flatten, Dense
from tf_agents.networks import Network
from tf_agents.networks import sequential

def normal_dqn(env):
  preprocessing_layer = keras.layers.Lambda(lambda obs: tf.cast(obs, np.float32) / 255.)
  conv_layer_params=[(32, (8, 8), 4), (64, (4, 4), 2), (64, (3, 3), 1)]
  fc_layer_params=[512]
  return QNetwork(
      env.observation_spec(),
      env.action_spec(),
      preprocessing_layers=preprocessing_layer,
      conv_layer_params=conv_layer_params,
      fc_layer_params=fc_layer_params
  )

def dueling_dqn(input_shape=(84, 84, 4), action_count=9):
  input = Input(shape=input_shape, name='input')
  preprocessing_layer = keras.layers.Lambda(lambda obs: tf.cast(obs, np.float32) / 255.)(input)
  conv_1 = Conv2D(filters=32, kernel_size=8, strides=4, padding='valid', activation='relu')(preprocessing_layer)
  conv_2 = Conv2D(filters=64, kernel_size=4, strides=2, padding='valid', activation='relu')(conv_1)
  conv_3 = Conv2D(filters=64, kernel_size=3, strides=1, padding='valid', activation='relu')(conv_2)
  flatten = Flatten()(conv_3)
  # branch state
  flatten_state = Dense(512, activation='relu')(flatten)
  state = Dense(1, activation='linear')(flatten_state)
  # branch advantages
  flatten_advantages = Dense(512, activation='relu')(flatten)
  raw_advantages = Dense(action_count,activation='linear')(flatten_advantages) 

  # action score - mean action score = centered action score
  advantages = raw_advantages - tf.reduce_max(raw_advantages, axis=1, keepdims=True) 
  Q_values = state + advantages
  model = keras.Model(inputs=[input], outputs=[Q_values])
  q_net = sequential.Sequential([model])
  return q_net

q_net = dueling_dqn(action_count=env.action_space.n) if dueling else normal_dqn(tf_training_env)

## DQN Agent

Qui viene inizializzato l'agente tramite le classi `DqnAgent` e `DdqnAgent`.
<p><img src='https://www.sitolorenzogardini.altervista.org/images/agent_creation.png' width='400'/></p>

In [53]:
from tf_agents.agents.dqn.dqn_agent import DqnAgent, DdqnAgent
train_step = tf.Variable(0)
update_period = 4 # train the model every 4 steps

optimizer = keras.optimizers.legacy.Adam(learning_rate = 2.5e-4, clipnorm = 1.0)

epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0, # initial ε
    decay_steps=1_000_000,
    end_learning_rate=0.01) # final ε

klass = DdqnAgent if double else DqnAgent
agent = klass(tf_training_env.time_step_spec(),
                 tf_training_env.action_spec(),
                 q_network=q_net,
                 optimizer=optimizer,
                 target_update_period=1_000,
                 td_errors_loss_fn=keras.losses.Huber(reduction="none"),
                 gamma=0.99, # discount factor
                 train_step_counter=train_step,
                 epsilon_greedy=lambda: epsilon_fn(train_step))
agent.initialize()

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......conv2d_2
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......flatten
.........vars
......input_layer
.........vars
......lambda
.........vars
......tf_op_lambda
.........vars
......tf_op_lambda_1
.........vars
......tf_op_lambda_2
.........vars
...vars
Keras model archive saving:
File Name                                             Modified             Size
variables.h5                                   2023-04-03 14:27:21     13209200
metadata.json                                  2023-04-03 14:27:21           64
config.json                                    2023-04-03 14:27:21 

## Buffer

Qui viene creato il buffer. La libreria non permette di usare Prioritized Experience Replay.
<p><img src='https://www.sitolorenzogardini.altervista.org/images/replay_
buffer_creation.png' width='400'/></p>

In [54]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer
# https://gist.github.com/Kenneth-Schroeder/6dbdd4e165331164e0d9dcc2355698e2
# from tf_prioritized_replay_buffer import TFPrioritizedReplayBuffer

'''
replay_buffer = TFPrioritizedReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)
'''

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=tf_training_env.batch_size,
    max_length=100_000)

2023-04-03 14:27:21.511515: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2822400000 exceeds 10% of free system memory.


## Observers

Qui vengono creati gli observers. Questi vengono notificati dal Driver con le _trajectories_.
<p><img src='https://www.sitolorenzogardini.altervista.org/images/observers.png' width='400'/></p>

In [55]:
from tf_agents.metrics import tf_metrics

replay_buffer_observer = replay_buffer.add_batch

training_metrics = [
  tf_metrics.NumberOfEpisodes(),
  tf_metrics.EnvironmentSteps(),
  tf_metrics.AverageReturnMetric(),
  tf_metrics.AverageEpisodeLengthMetric(),
]

## Driver

I _Drivers_ campionano, in base alla policy specificata, le _trajectories_ dagli _environments_. Vengono creati due `DynamicStepDriver`:
- `init_driver`: utilizza una `RandomTFPolicy` per inizializzare la reply memory
- `collect_driver`: utilizza la policy del modello per eseguire le azioni nell'env e collezionare *trajectories*

<p><img src='https://www.sitolorenzogardini.altervista.org/images/driver_creation.png' width='400'/></p>

In [56]:
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver

#initialization
initial_collect_policy = RandomTFPolicy(tf_training_env.time_step_spec(),
                                        tf_training_env.action_spec())
init_driver = DynamicStepDriver(
    tf_training_env,
    initial_collect_policy,
    observers=[replay_buffer_observer],
    num_steps=20_000)

# policy
collect_driver = DynamicStepDriver(
    tf_training_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + training_metrics,
    num_steps=update_period) # collect 4 steps for each training iteration

## Dataset

Genera i dati attingendo dal _replay buffer_, li prefetcha per velocizzare l'addestramento. Qui `num_steps` viene messo a due in modo da fare il sampling di due _trajectories_ consegutive. Questo perchè una trajectory possiede solamente lo stato attuale ma non quello successivo, che invece è richiesto da DQN per addestrarsi.
<p><img src='https://www.sitolorenzogardini.altervista.org/images/dataset_creation.png' width='400'/></p>

In [57]:
dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2,
    num_parallel_calls=3).prefetch(3)

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


## Training

Le funzioni `run` dei driver e `train` dell'agente vengono vrappate con `function` di tensorflow in modo che possano essere ottimizzate

In [58]:
from tf_agents.utils.common import function
collect_driver.run = function(collect_driver.run)
init_driver.run = function(init_driver.run)
agent.train = function(agent.train)

Qui la reply memory viene inizializzata campionando casualmente l'environment

In [59]:
# init memory
print('replay memory init')
final_time_step, final_policy_state = init_driver.run()

replay memory init


Funzione di addestramento della rete. Il corpo corrisponde al classico algoritmo di apprendimento di DQN:
1) si collezionano steps fino a che non bisogna aggiornare la rete
2) si aggiorna la rete
3) si ripetono i passi 1 e 2 fino a che non vengono effettuate le interazioni specificate da `n_interations`

In [60]:
def train_agent(n_iterations):
  time_step = None
  policy_state = agent.collect_policy.get_initial_state(tf_training_env.batch_size)
  iterator = iter(dataset)
  for iteration in range(n_iterations):
    time_step, policy_state = collect_driver.run(time_step, policy_state)
    trajectories, _ = next(iterator)
    agent.train(trajectories)
    if iteration % 100 == 0:
      log_metrics(training_metrics)

In [61]:
train_agent(10_000)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


2023-04-03 14:28:37.193757: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 4
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0
INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 404
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0
INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 804
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0
INFO:absl: 
		 NumberOfEpisodes = 1
		 EnvironmentSteps = 1204
		 AverageReturn = -21.0
		 AverageEpisodeLength = 807.0
INFO:absl: 
		 NumberOfEpisodes = 1
		 EnvironmentSteps = 1604
		 AverageReturn = -21.0
		 AverageEpisodeLength = 807.0
INFO:absl: 
		 NumberOfEpisodes = 2
		 EnvironmentSteps = 2004
		 AverageReturn = -20.5
		 AverageEpisodeLength = 863.0
INFO:absl: 
		 NumberOfEpisodes = 2
		 EnvironmentSteps = 2404
		 AverageReturn = -20.5
		 AverageEpisodeLength = 863.0
INFO:absl: 
		 NumberOfEpisodes = 3


INFO:absl: 
		 NumberOfEpisodes = 26
		 EnvironmentSteps = 23604
		 AverageReturn = -20.200000762939453
		 AverageEpisodeLength = 871.7000122070312
INFO:absl: 
		 NumberOfEpisodes = 26
		 EnvironmentSteps = 24004
		 AverageReturn = -20.200000762939453
		 AverageEpisodeLength = 871.7000122070312
INFO:absl: 
		 NumberOfEpisodes = 27
		 EnvironmentSteps = 24404
		 AverageReturn = -20.200000762939453
		 AverageEpisodeLength = 872.5
INFO:absl: 
		 NumberOfEpisodes = 27
		 EnvironmentSteps = 24804
		 AverageReturn = -20.200000762939453
		 AverageEpisodeLength = 872.5
INFO:absl: 
		 NumberOfEpisodes = 27
		 EnvironmentSteps = 25204
		 AverageReturn = -20.200000762939453
		 AverageEpisodeLength = 872.5
INFO:absl: 
		 NumberOfEpisodes = 28
		 EnvironmentSteps = 25604
		 AverageReturn = -20.100000381469727
		 AverageEpisodeLength = 889.2999877929688
INFO:absl: 
		 NumberOfEpisodes = 28
		 EnvironmentSteps = 26004
		 AverageReturn = -20.100000381469727
		 AverageEpisodeLength = 889.2999877929688


# Plots

In [63]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statistics
import seaborn as sns
from pathlib import Path

plt.rcParams["figure.figsize"] = (15,5)

## Utility functions

`load_csv(path, skiprows)` carica from `path` il file _csv_ saltando le prime `skiprows` righe

In [64]:
def load_csv(path, skip_rows=1):
  return pd.read_csv(path, skiprows=skip_rows, sep=',').r.values

`compute_moving_avg_rewards(rewards, moving_avg_window_size)` calcola la media mobile dell'array delle rewards `rewards` con una finestra grande `moving_avg_windows_size` 

In [65]:
def compute_moving_avg_rewards(rewards, moving_avg_window_size=100):
  return np.convolve(rewards, np.ones(moving_avg_window_size)/moving_avg_window_size, mode='valid')

`plot_rewards(data, label, limit_data=5000)` grafica le rewards specificate in `data` con la corrispondente `label`, limitando i dati con `limit_data`

In [66]:
def plot_rewards(data, label):
  plt.plot(data, label=[label])
  plt.xlabel('Episodes')
  plt.ylabel('Rewards')

`first_time_value_reached(data, value)` restituisce l'indice della posizione in cui `value` è >= di `data[i]`, `min_threshold` serve per evitare che ci siano valori molto alti di media all'inizio

In [67]:
def first_time_value_reached(data, value):
  index = np.argmax(data > value)
  return None if index == 0 else index

`compute_speedup(initial_value, final_value)` calcola in percentuale l'incremento di `initial_value` rispetto a `final_value` 

In [68]:
def compute_speedup(value_1, value_2):
  if value_1 is None or value_2 is None:
    return np.inf
  min_value = min(value_1, value_2)
  if min_value == 0:
    return np.inf
  return round( abs(value_1 - value_2) / min_value * 100, ndigits=2)

`plot_together(list_of_csv, moving_avg_windows_size=100, limit_data=5000, first_value_reached=20)` grafica l'elenco specificato di csv in `list_of_csv`, computa di ciascuno di essi la moving average utilizzando come finestra `moving_avg_windows_size` e limitando i dati a `limit_data` elementi. 


In [69]:
def plot_together(list_of_csv, limit_data=5000, moving_avg_windows_size=100, first_value_reached=20):
  csvs = [load_csv(csv)[:limit_data] for csv in list_of_csv]
  algos = [Path(csv).stem for csv in list_of_csv]
  moving_avgs = np.array([compute_moving_avg_rewards(csv, moving_avg_windows_size) for csv in csvs])
  data = {algo : averages for algo, averages in zip(algos, moving_avgs)}
  indexes = {algo : first_time_value_reached(avg, first_value_reached) for algo, avg in zip(algos, moving_avgs)}
  sorted_indexes = dict(sorted(indexes.items(), key=lambda x: (x[1] is None, x[1])))
  color_seq = plt.rcParams['axes.prop_cycle'].by_key()['color']

  print(f"Moving average over {first_value_reached} for the first time:")
  for index, algo in sorted_indexes.items():
    print(algo, index)
  print()

  print("Speed up:")
  for i, (algo, faster_avg) in enumerate(list(sorted_indexes.items())[:-1], start=1):
    print(algo)
    for algo, slower_avg in list(sorted_indexes.items())[i:]:
      speed_up = compute_speedup(faster_avg, slower_avg)
      print(f'\t{algo}: {speed_up}%')

  for algo, moving_avg in data.items():
    plt.title('All models rewards')
    plt.xlabel('Episodes')
    plt.ylabel('Rewards')
    plt.plot(moving_avg)
  
  plt.legend(data.keys())

  for color, index in zip(color_seq, indexes.values()):
    if index is not None:
      plt.axvline(x = index, color = color, linestyle = 'dashed')
      plt.annotate(index, (index, first_value_reached))

  plt.axhline(20, color = 'gray', linestyle = 'dashed')

`compare_last_n_games(list_of_csv, lasts_n_games=50)` mosta un confronto tra gli ultimi `lasts_n_games` caricando le rewards da `list_of_csv`

In [70]:
def compare_last_n_games(list_of_csv, lasts_n_games=50):
  mean_last_n_games = [np.mean(load_csv(csv)[-lasts_n_games:]) for csv in list_of_csv]
  dataframe = pd.DataFrame({
      'average_score' : mean_last_n_games,
      'algorithm' : [Path(csv).stem for csv in list_of_csv]
  }).sort_values('average_score', ascending=False)
  ax = sns.barplot(data=dataframe, x=f'average_score', y='algorithm')
  ax.bar_label(ax.containers[0])

`filter_csv(regex='')` carica i _csv_ in base alla regex passata (N.B. quest'ultima non deve includere `.csv`)

In [71]:
def filter_csv(regex=''):
  return glob(regex + '.csv')

## Download

Download delle rewards

In [ ]:
!wget  https://sitolorenzogardini.altervista.org/csvs/rewards.zip
!wget  https://sitolorenzogardini.altervista.org/csvs/fails.zip

!unzip rewards.zip
!unzip fails.zip

!rm rewards.zip
!rm fails.zip

## Plots

Caricamento per categoria dei file di configurazione

In [ ]:
stable_baselines_files = filter_csv('sb3_*')
stable_baselines_ok = [f'sb3_{algo}.csv' for algo in ['A2C_16', 'PPO_8', 'DQN_1']]
stable_baselines_bad = [f'sb3_{algo}.csv' for algo in ['A2C_1', 'PPO_1', 'DQN_16']]
fails = filter_csv('fail_*')
tf_agents = filter_csv('tf_*')

Plot dei migliori risultati ottenuti utilizzando Stable Baselines 3

In [ ]:
plot_together(stable_baselines_ok, 8_000)

Plot degli algoritmi utilizzando gli algoritmi con un numero scorretto di evironments

In [ ]:
plot_together(stable_baselines_bad, 8_000)

Confronto tra tutti gli algoritmi di Stable Baselines 3

In [ ]:
plot_together(stable_baselines_files, 8_000)

Plot di tutti gli algoritmi che non hanno portato buoni risultati causati dallo `StickyActionEnv`

In [ ]:
plot_together(fails, 10_000)

Plot dei risultati ottenuti con Tensorflow Agents

In [ ]:
plot_together(tf_agents, 6_000)

Plot del confronto tra i migliori risultati tra Stable Baselines 3 e Tensorflow Agents

In [ ]:
plot_together(stable_baselines_ok + ['tf_DDQN.csv', 'tf_DQN.csv'], limit_data=6000)

Media raggiunta nelle ultime N partite da ciascun algoritmo a confronto

In [ ]:
compare_last_n_games(glob('*.csv'), lasts_n_games=50)